# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

In this exercise we will need to compute the derivatives with respect to all parameters of the following function:

$$f(a, b, c) = -a^{3} + sin(3b) - \frac{1}{c} + b^{2.5} - a^{0.5}$$

By using simple rules of calculus, it is easy to calculate analytical gradients of $f$ as follows:

$$\frac{df}{da} = -3a^{2} - 0.5a^{-0.5}$$
$$\frac{df}{db} = 3 cos(3b) + 2.5b^{1.5}$$
$$\frac{df}{dc} = c^{-2}$$

Now we can use these formulas to compute all gradients required in code:

In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
    """Computes gradients with respect to all parameters."""
    # Computing the derivate of 'f' with respect to 'a'
    df_da = -3 * a**2 - 0.5 * a**(-0.5)
    # Computing the derivate of 'f' with respect to 'b'
    df_db = 3 * cos(3*b) + 2.5 * b**(1.5)
    # Computing the derivate of 'f' with respect to 'c'
    df_dc = c**(-2)
    
    return [df_da, df_db, df_dc] # returns [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
    ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


The next task is to compute the gradient numerically by using the definition of the derivative:

$$f'(a) = \lim_{h \rightarrow 0} \frac{f(a + h) - f(a)}{h}$$

It means that by considering smaller and smaller values of $h$, we will get better and better approximations of the derivate at some point $a$. Certainly, if we take a very small value of $h$ (for instance, $h = 10^{-100}$), gradients will turn to zero, since the difference in the numerator will become zero. Thus, we will take some sensible value for $h$. Since the function takes 3 parameters as input, we will consider the point $(a = 2, b = 3, c = 4)$, which has been chosen to check the correctness of the exercise.

In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# Selecting the step value
h = 1e-6

# Defining points at which we will compute numerical gradients
a = 2
b = 3
c = 4

# Computing gradients with respect to 'a', 'b', 'c' numerically
df_da = (f(a + h, b, c) - f(a, b, c)) / h
df_db = (f(a, b + h, c) - f(a, b, c)) / h
df_dc = (f(a, b, c + h) - f(a, b, c)) / h
# -----------
numerical_grad = [df_da, df_db, df_dc] # assigns values of gradients
# -----------

for dim in range(3):
    ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


It can be seen that for the point $(2, 3, 4)$, the convergence has been attained by using $h = 10^{-6}$. It can also be checked that using higher $h$ was always resulting in divergence of the gradient.

Now, we should use an alternative formula for computing derivatives. It looks as follows:

$$f'(a) = \lim_{h \rightarrow 0} \frac{f(a + h) - f(a - h)}{2h}$$

We will check if this method indeed provides a better numerical approximation of the derivative:

In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# Selecting the step value
h = 1e-6

# Defining points at which we will compute numerical gradients
a = 2
b = 3
c = 4

# Computing gradients with respect to 'a', 'b', 'c' numerically (Symmetric gradient method)
df_da = (f(a + h, b, c) - f(a - h, b, c)) / (2 * h)
df_db = (f(a, b + h, c) - f(a, b - h, c)) / (2 * h)
df_dc = (f(a, b, c + h) - f(a, b, c - h)) / (2 * h)

# -----------
numerical_grad2 = [df_da, df_db, df_dc]
# -----------

for dim in range(3):
    ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


It is not immediately seen if the symmetric derivative method improves upon the usual derivative formula but it can also be checked that already at $h = 10^{-3}$ the gradients converge which is not the case in the previous cell. Besides, if we take a look at $df/dc$, it is seen to be better approximated.

## section 2: support for softmax

In [5]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        """Initializes the object of Value class."""
        # Attribute for storing values
        self.data = data
        # Attribute for storing computed gradients
        self.grad = 0.0
        # Attribute for storing backpropagation function
        self._backward = lambda: None
        # Attribute for storing children nodes in the computational graph
        self._prev = set(_children)
        # Attribute for storting a label for the mathematical operation
        self._op = _op
        # Attribute for storting a label of the Value instance
        self.label = label
        
    def __repr__(self):
        """Returns string representation of the object."""
        
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        """Adds values of two instances together."""
        # Convering a number to Value object if needed
        other = other if isinstance(other, Value) else Value(other)
        # Addition operation
        out = Value(self.data + other.data, (self, other), '+')
        # Computes gradients for the addition operation
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __radd__(self, other):
        """Implements reversal addition operation."""
        
        return self + other
    
    def __mul__(self, other):
        """Multiplies values of two instances together."""
        # Convering a number to Value object if needed
        other = other if isinstance(other, Value) else Value(other)
        # Multiplication operation
        out = Value(self.data * other.data, (self, other), "*")
        # Computes gradients for the multiplication operation
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other):
        """Implements reversal multiplication operation."""
        
        return self * other
    
    def __neg__(self):
        """Implements taking a negative value."""
        
        return self * (-1)
    
    def __pow__(self, other):
        """Computes value of one instance to the power of the value of another instance."""
        # Making sure that the values supplied are integer or floating point number
        assert isinstance(other, (int, float)), "Only integer and floating point powers are supported"
        # Power operation
        out = Value(self.data**other, (self,), f'**{other}')
        # Computes gradient for the power operation
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __truediv__(self, other):
        """Implements a division operation."""
        
        return self * other**(-1)
    
    def __sub__(self, other):
        """Implements a subtraction operation."""
        
        return self + (-other)
    
    def __rsub__(self, other):
        """Implements a reversal subtraction operation"""
        
        return self + (-other)
    
    def exp(self):
        """Implements the operation of taking an exponent."""
        # Exponent of some power operation
        x = self.data
        out = Value(exp(x), (self, ), 'exp')
        # Computing gradient for the exponent operation
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def log(self):
        """Implements an operation of taking a natural logarithm."""
        # Natural logarithm operation
        x = self.data
        out = Value(log(x), (self, ), 'log')
        # Computing gradient for the natural logarithm operation
        def _backward():
            self.grad += (1/x) * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        """Implements a backpropagation via topological sorting."""
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

Using the `Value` object implemented in the video, I have added and reimplemented some methods of the class above in order to be able to accurately perform forward pass/backpropagation using $NLL$ loss. For all necessary intermediary mathematical operations used in computing the loss, specific methods have been written along with the respective `_backward()` submethod for doing backpropagation and storing/accumulating computed gradients using a chain rule. 

It can now be seen that the gradients obtained are very close to those expected:

In [6]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
    """Implements a Softmax activation function."""
    # Exponentiating all inputs
    counts = [logit.exp() for logit in logits]
    # Computing probabilities
    denominator = sum(counts)
    out = [c / denominator for c in counts]
    
    return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


We can replicate the same results by using the tools of `torch` library.

In [7]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

In [8]:
# Defining a PyTorch tensor
logit_tensor = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()

# Enabling computing gradients
logit_tensor.requires_grad = True

logit_tensor

tensor([ 0.,  3., -2.,  1.], dtype=torch.float64, requires_grad=True)

In [9]:
# Computing an NLL loss (forward pass)
loss = -F.log_softmax(logit_tensor, dim=-1)[3]

In [10]:
# Backpropagation (backward pass)
loss.backward()

In [11]:
print(loss.data.item())

2.1755153626167143


In [12]:
logit_tensor.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)

It can be seen that the gradients obtained are very close to the required ones.